In [1]:
import pandas as pd
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import xml.etree.ElementTree as ET
from lxml import etree
from scipy.sparse import hstack
import numpy as np
import warnings
from preprocessing.xml_2_dataframe import Xml2DataFrame


In [2]:
path_train = r'./data/Laptops_Train_v2.xml'
path_test = r'./data/Laptops_Test_Gold.xml'
new_test_path = r'./data/test.xml'


In [4]:
# xml parser
def get_xml_data(path):
    xml2df = Xml2DataFrame()
    xml_dataframe = xml2df.process_data(path_train)
    xml_dataframe.head()


get_list(path_train)

/home/avi/github/aspect-term-extractor
     id                                               text  \
0  2339  i charge it at night and skip taking the cord ...   
1   812  i bought a hp pavilion dv4-1222nr laptop and h...   
2  1316  the tech guy then said the service center does...   
3  2328  i investigated netbooks and saw the toshiba nb...   
4  2193  the other day i had a presentation to do for a...   

                                         aspect_info  
0  [{'to': '45', 'polarity': 'neutral', 'term': '...  
1                                               None  
2  [{'to': '41', 'polarity': 'negative', 'term': ...  
3                                               None  
4                                               None  
